In [1]:
from slf_dataset import SLFDataset1bit, DEFAULT_ROOT
import matplotlib.pyplot as plt
import torch
import os
from params import DATA_PATH as ROOT
train_set = SLFDataset1bit(root_dir=os.path.join(ROOT, 'slf_mat'), 
                    csv_file=os.path.join(ROOT, 'details.csv'), total_data=10000, sample_size=[0.05, 1])


ImportError: cannot import name 'SLFDataset1bit'

In [ ]:
max_iter = 500

def F(y):
    return 1/(1+torch.exp(-y))
    
def dither(y):
    F_y = F(y)
    return torch.bernoulli(F_y)

class BaleviLoss(torch.nn.Module):
    """
    Total variation loss.
    """
    def __init__(self, lambda_reg = 0.5):
        super(BaleviLoss, self).__init__()
        self.lambda_reg = lambda_reg
        
    def forward(self, y_hat, y_target):
        quant_loss = -self.lambda_reg*((y_target*y_hat).sum()) + torch.norm(y_hat[0,0,:,:], p='nuc') + torch.norm(y_hat[0,1,:,:], p='nuc')
        return quant_loss

    
def run_descent(generator, y, mean, y_true, loss_criterion='bce', print_out=False, true=None, z_dimension=32, lr=0.01):
    z = torch.randn((1,z_dimension), dtype=torch.float32)

    mse_criterion = nn.MSELoss()
    if loss_criterion == 'balevi':
        criterion = BaleviLoss()
        loss_change_threshold = 0.0001
    elif loss_criterion == 'bce':
        criterion = nn.BCELoss()
        loss_change_threshold = 0.0000001
    # First select a good random vector
    min_criterion = 9999999
    for i in range(400):
        temp = torch.randn((1,z_dimension), dtype=torch.float32)
        temp_out = generator(temp)
#         temp_out = temp_out.permute(0,1,3,2)
        y_hat = temp_out - mean
        
        if loss_criterion == 'bce':
            y_hat = F(y_hat) 
        
        temp_criterion = criterion(y_hat, y) 
        if  temp_criterion < min_criterion:
            z.data = temp.clone()
            min_criterion = temp_criterion
            print('min_first', min_criterion.item())

    for i in range(200):
        temp = 0.2*torch.randn((1,z_dimension), dtype=torch.float32) + z
        temp_out = generator(temp)
#         temp_out = temp_out.permute(0,1,3,2)
        y_hat = temp_out - mean
        
        if loss_criterion == 'bce':
            y_hat = F(y_hat)     
    
        temp_criterion = criterion(y_hat, y) 
        if  temp_criterion < min_criterion:
            z.data = temp.clone()
            min_criterion = temp_criterion
            print('min_second', min_criterion.item())

    z.requires_grad = True
    optimizer = torch.optim.Adam([z], lr=0.01)
    
    num_iter = 0
    loss_prev = 999999
    loss_current = 999
    while num_iter < max_iter and (loss_prev - loss_current > loss_change_threshold):
        print(loss_prev-loss_current)
        loss_prev = loss_current
        optimizer.zero_grad()
        
        gen_out = generator(z)
#         gen_out = gen_out.permute(0,1,3,2)
#         y_hat = torch.matmul(A, torch.matmul(gen_out, P).reshape(-1))
        y_hat = gen_out - mean
        
        if loss_criterion == 'bce':
            y_hat = F(y_hat) 
        
        loss = criterion(y_hat, y)
        
        loss.backward()
        optimizer.step()

        if print_out:
            clear_output(wait=True)
#             if true is not None:
#                 actual_loss = criterion(gen_out.squeeze(), true.squeeze())
#                 print(lr, loss.item(), actual_loss.item())
            actual_loss = torch.norm(gen_out-y_true)/torch.norm(y_true)
            print(actual_loss.item())
#             plt.imshow(gen_out[0,0,:,:].detach().squeeze().numpy())
            print(num_iter)
            print(loss_prev - loss.item())
            
            plt.show()
            time.sleep(0.01)
        loss_current = loss.item()
        num_iter += 1
#     actual_loss = criterion(gen_out, true)
    
    return gen_out